In [26]:
!pip install datasets transformers attrdict

In [27]:
import torch
import torch.nn as nn
from transformers import BertPreTrainedModel, BertModel
from transformers import BertTokenizer
from typing import Union, Optional

import numpy as np
from transformers.pipelines import ArgumentHandler
from transformers import (
    Pipeline,
    PreTrainedTokenizer,
    ModelCard
)
from pprint import pprint
import os
import google.colab as colab
import pandas as pd
import json
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import random

In [28]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

ROOT_DIR =  mount_google_drive() + "/goemotion/"
CHECKPOINT_ROOT = ROOT_DIR+ "goemotion-ckpt/original/checkpoint-10000"

Mounted at /content/gdrive


In [29]:
test_df = pd.read_csv(ROOT_DIR + "data/original/test.tsv", sep='\t', header=None, names=["sentence", "label", "encoding"])
test_sentences = test_df.sentence.tolist()

In [30]:
# Load T5 mapping
with open(ROOT_DIR + "t5.json", 'r') as f:
  t5_map = json.load(f)

In [31]:
test_df

,sentence,label,encoding
0,I’m really sorry about your situation :( Altho...,25,eecwqtt
1,It's wonderful because it's awful. At not with.,0,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1
4,They got bored from haunting earth for thousan...,27,eem5uti
...,...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,15,efeeasc
5423,Well that makes sense.,4,ef9c7s3
5424,Daddy issues [NAME],27,efbiugo
5425,So glad I discovered that subreddit a couple m...,0,efbvgp9


In [32]:
candidate_sentences = [t5_map[x][random.randint(0, len(t5_map[x]) - 1)] for x in test_sentences]

In [33]:
new_df = test_df.copy()

In [34]:
new_df["sentence"] = candidate_sentences

In [35]:
new_df

,sentence,label,encoding
0,I’m really sorry about your situation :( altho...,25,eecwqtt
1,Is it wonderful because it's awful. At not with.,0,ed5f85d
2,"What are the chances of a Kings win, good luck...",13,een27c3
3,Thank you for teaching me! I was surprised to ...,15,eelgwd1
4,Had been a ghost for thousands of years. They ...,27,eem5uti
...,...,...,...
5422,How can I get rid of BP 1 after hospitalization?,15,efeeasc
5423,Why does this make sense?,4,ef9c7s3
5424,Daddy issues [NAME] in U.S.C.A.G.E.,27,efbiugo
5425,How fun is to visit reddit with other authors ...,0,efbvgp9


In [36]:
test_df

,sentence,label,encoding
0,I’m really sorry about your situation :( Altho...,25,eecwqtt
1,It's wonderful because it's awful. At not with.,0,ed5f85d
2,"Kings fan here, good luck to you guys! Will be...",13,een27c3
3,"I didn't know that, thank you for teaching me ...",15,eelgwd1
4,They got bored from haunting earth for thousan...,27,eem5uti
...,...,...,...
5422,Thanks. I was diagnosed with BP 1 after the ho...,15,efeeasc
5423,Well that makes sense.,4,ef9c7s3
5424,Daddy issues [NAME],27,efbiugo
5425,So glad I discovered that subreddit a couple m...,0,efbvgp9


In [38]:
new_df.to_csv(ROOT_DIR + "data/new.tsv", sep='\t', index=False, header=False)